Run this file after running "1_Clean_Eora.ipynb"

This file is used to generate paratmers used for estimating trade cost and running models

# Set up

In [1]:
import os
import pandas as pd 
import numpy as np


In [2]:
# Set working directory
wd = os.path.expanduser("~/Dropbox/Tariff_Project")
os.chdir(wd)
print(f"Current working directory: {os.getcwd()}")

# 3. Define data paths and parameters
# Path to raw Eora BP data
data_path = os.path.join(wd, "3_Result/eora_clean")


save_path = os.path.join(wd, "3_Result/parameters")


# Verify paths
print(f"Cleaned Eora data folder: {data_path}")

Current working directory: /Users/lishuangcen/Dropbox/Tariff_Project
Cleaned Eora data folder: /Users/lishuangcen/Dropbox/Tariff_Project/3_Result/eora_clean


# Part I ： Run this part after running "1_Clean_Eora.ipynb"

In [3]:
# ----------------------------
#  Read country / sector lists
# ----------------------------
country_list = (
    pd.read_csv(os.path.join(data_path, "country_list.csv"), header=0)["0"]
    .astype(str)
    .tolist()
)

sector_list = (
    pd.read_csv(os.path.join(data_path, "sector_list.csv"), header=0)["0"]
    .astype(str)
    .tolist()
)

label_list = pd.read_csv(os.path.join(data_path, "labels.csv"), header=0)

# ----------------------------
#  Basic dimension information
# ----------------------------
N = len(country_list)  # number of countries
J = len(sector_list)   # number of sectors

print(f"{N = }, {J = }")


N = 37, J = 25


In [4]:
# -------------------------------------------------
#  Read input‑output tables, final demand, value added
# -------------------------------------------------

io = (
    pd.read_csv(os.path.join(data_path, "T_final_2017.csv"), header=0)
    .fillna(0)           
    .to_numpy(float)      
)

fd = (
    pd.read_csv(os.path.join(data_path, "FD_final_2017.csv"), header=0)
    .fillna(0)
    .to_numpy(float)
)



va = (
    pd.read_csv(os.path.join(data_path, "VA_final_2017.csv"), header=0)
    .fillna(0)
    .to_numpy(float)
)

print(f"io shape: {io.shape}, fd shape: {fd.shape}, va shape: {va.shape}")

io shape: (925, 925), fd shape: (925, 222), va shape: (6, 925)


## alpha(N,J): Calibrate $\alpha_n^{j}$ (preference paramter) in household utility function 


In [5]:
# Final demand accounts include: 
# 1.Household final consumption 
# 2.Non-profit institutions serving households
# 3.Government final consumption 
# 4.Gross fixed capital formation
# 5.Changes in inventories 
# 6.Acquisitions less disposals of valuables

# Here I exclude 5 and 6 to better represent household preference
fd_n = 6 
fd_n_2 = 4
keep_fd_n = [0, 1, 2, 3]  

# Step 1: select the four accounts (exclude the last two)
cols_to_select = []
for c_idx in range(N):
    base = c_idx * fd_n
    cols_to_select.extend([base + a for a in keep_fd_n])

fd_select = fd[:, cols_to_select]      # (N*J, 4*N)

# Step 2：sum up over accounts for final demand 
fd_sum = fd_select.reshape(fd_select.shape[0], N, 4).sum(axis=2) # reshape to (NJ, N, 4 and sum up over the last dim)

# Step 3: sum up over countries for the same sector
fd_sum_reshaped = fd_sum.reshape(N, J, N)
fd_final = fd_sum_reshaped.sum(axis=0)         # (J, N)：（j,n）is country n's total comsumption in sector j, no matter where it comes from 

# ----------- correct above - checked by hand --------------

# Step 4: calculate shares
total_fd = fd_final.sum(axis=0, keepdims=True) # (1, N) # sum up over all sectors for each country, that is, each country's total final demand (total final consumption)
Alpha_jn = fd_final / total_fd                 # (J, N) 
Alpha_nj = Alpha_jn.T                          # (N, J)

# Step 5: adjust
Alpha_nj = np.clip(Alpha_nj, 0.00001, 0.99999)
Alpha_nj = Alpha_nj / Alpha_nj.sum(axis=1, keepdims=True)

# ----------- correct above - checked by hand --------------



In [6]:
# Check if it satisfies the model

alpha = Alpha_nj.copy()  # alpha is the final variable name we use in model simulation

# Check
# Check 1: For each n, the sum over j equals 1
sum_by_n = np.sum(alpha, axis=1)
check_sum = np.allclose(sum_by_n, 1, atol=1e-6)  # Allowing a small numerical tolerance
if check_sum:
    print("For each country, the sum over secotrs equals 1 ✅")
else:
    print("There are some country where the sum over sectors is not 1 ❌")
    print("The issue occurs at the following n indices:", np.where(np.abs(sum_by_n - 1) > 1e-6)[0])
    print("The corresponding sums are:", sum_by_n[np.where(np.abs(sum_by_n - 1) > 1e-6)[0]])
# Check 2: Every value in alpha is between 0 and 1
check_range = np.all((alpha >= 0) & (alpha <= 1))
if check_range:
    print("Every value in alpha is between 0 and 1 ✅")
else:
    print("There are values in alpha that are not between 0 and 1 ❌")
    print("These values are at positions:", np.where((alpha < 0) | (alpha > 1)))
    print("The values are:", alpha[(alpha < 0) | (alpha > 1)])



For each country, the sum over secotrs equals 1 ✅
Every value in alpha is between 0 and 1 ✅


In [7]:
# Save to csv, for check
alpha_df = pd.DataFrame(Alpha_nj,
                        index=country_list,
                        columns=sector_list)


#os.makedirs(save_path, exist_ok=True) 
alpha_df.to_csv(os.path.join(save_path, "Alpha_nj_2017.csv"))

print("Alpha_nj_2017.csv saved successfully.")

Alpha_nj_2017.csv saved successfully.


## GO(N,J): Calculate Gross Output of Intermediate Goods, Final Goods, and All Goods


In [8]:
# Total output: sum rows
io_GO = io.sum(axis=1)         # (NJ,1) # intermediate output: sum up for each row
fd_GO = fd_select.sum(axis=1)  # (NJ,1) # final output:sum up for each row. Previously, I incorrectely wrote fd_GO = fd.sum(axis=1)  

GO_temp    = io_GO + fd_GO         # (NJ,)

GO = GO_temp.reshape(N, J, order="C")  # (N,J) # row - country, col - sector

go_file = os.path.join(save_path, "GO_2017.csv")
pd.DataFrame(GO, index=country_list, columns=sector_list).to_csv(go_file)

##  $D_n$: Trade Deficit 

In [9]:
# D_n = IM - EX

# sum up over sectors for total output
EX = GO.sum(axis = 1) 


# sum up over sectors / fd accountrs for total consumption / imports
io_CS = io.sum(axis = 0) # sum up over all import sources
C_io = io_CS.reshape(N, J).sum(axis=1) 
fd_CS = fd_select.sum(axis = 0)  # sum up over all import sources
C_fd = fd_CS.reshape(N, fd_n_2).sum(axis=1)  

IM = C_io + C_fd

D = IM - EX



df_trade_deficit = pd.DataFrame({
    "Country": country_list,
    "IM-EX": D
})

df_trade_deficit.to_csv("3_Result/parameters/D.csv", index=False, encoding="utf-8")


## $\beta(N,J)$: Calibrate $\gamma_n^j$ (value-added share) in production function 

In [10]:
# Value added accounts include: 1.Compensation of employees; 2.Taxes on production; 4.Net operating surplus; 5. Net mixed income; 6. Consumption of fixed capital
# Value added accounts exclude: 3. Subsidies on production
# Reason: for some countries, Subsidies on production is large negative number, resulting in the sum of VA negative

va_total = np.delete(va, 2, axis=0).sum(axis=0)   # (N·J,)

Gamma_vec = va_total / GO_temp                        # (N·J,)
Gamma_nj  = Gamma_vec.reshape(N, J, order="C")   # # row - country, col - sector


gt_one = Gamma_nj > 1
if gt_one.any():
    print(f"There are {gt_one.sum()} values greater than 1. They have been replaced with 0.99.")
    Gamma_nj[gt_one] = 0.99
else:
    print("No values are greater than 1.")

lt_zero = Gamma_nj < 0
if lt_zero.any():
    print(f"There are {lt_zero.sum()} values less than 0. They have been replaced with 0.01.")
    Gamma_nj[lt_zero] = 0.01
else:
    print("No values are less than 0.")

# Since further adjustment may be needed, DO NOT save it now; save after calculating gamma_njk

There are 38 values greater than 1. They have been replaced with 0.99.
No values are less than 0.


/var/folders/j3/y9kg_6px2gb9b4k8rrxcdxvr0000gn/T/ipykernel_59023/1716875595.py:7: RuntimeWarning: invalid value encountered in divide
  Gamma_vec = va_total / GO_temp                        # (N·J,)


## gamma(N,J,J): Calibarte $\gamma_n^{j,k}$

Note: gamma(n,j,k) refers to the share of input from sector k in the production of nj 

In [11]:
# Step 1: sum up over sectors
# Sum up each sectors. eg. sector 1 = sum of sector 1 of each countries (row1, row1+J, row1+2J...)
# The aim is to sum up all goods from the same sector across countries (no matter where they from)

io_sector_sum = io.reshape(N, J, -1).sum(axis=0)      # (J, N*J) # first reshape to (N,J,NJ),then, sum up over N, that is ,sum up across sectors no matter where they comes from 

# Step 2: Calculate Total Input (column sums)
total_input = io.sum(axis=0)                          # (N*J,)

# Step 3: Gamma_k_nj: row are the input sectors; colums are user "country-sector"
with np.errstate(divide="ignore", invalid="ignore"):
    Gamma_k_nj = io_sector_sum / total_input          # (J, N*J) 
Gamma_nj_k = Gamma_k_nj.T                             # (N*J, J) row - used by; col - from sector

# Step 4: Check if Gamma_nj_k contains any NA values
nan_mask = np.isnan(Gamma_nj_k)
if nan_mask.any():
    rows_with_nan = np.where(nan_mask.any(axis=1))[0]   
    n_idx = rows_with_nan // J
    j_idx = rows_with_nan %  J
    Gamma_nj[n_idx, j_idx] = 0.9999
    Gamma_nj_k[nan_mask] = 0.0001
    print("NA values have been handled.")
else:
    print("No NA values found in Gamma_nj_k.")

# NA appears because input of "KOR - Recycling", "ESP - Others" are all zeros across all coutries and sectors

# Step 5: Check and handle values less than 0 or greater than 1 in Gamma_nj_k
lt_zero = Gamma_nj_k < 0
if lt_zero.any():
    rows = np.where(lt_zero.any(axis=1))[0]
    n_idx = rows // J
    j_idx = rows %  J
    Gamma_nj_k[lt_zero] = 0.0001
    Gamma_nj[n_idx, j_idx] = 0.9999
    handled = True
else:
    handled = False

gt_one = Gamma_nj_k > 1
if gt_one.any():
    rows = np.where(gt_one.any(axis=1))[0]
    n_idx = rows // J
    j_idx = rows %  J
    Gamma_nj_k[gt_one] = 0.9999
    Gamma_nj[n_idx, j_idx] = 0.0001
    handled = True

print("Values less than 0 or greater than 1 have been handled." if handled
    else "No values less than 0 or greater than 1 found in Gamma_nj_k.")

# Step 6: Adjust to meet the model
row_sum = Gamma_nj_k.sum(axis=1)             # (N*J,)
row_rest = 1.0 - Gamma_nj.ravel(order="C")   # (N*J,)


scale = row_rest / row_sum  
Gamma_nj_k *= scale[:, None] # (NJ,J)



NA values have been handled.
No values less than 0 or greater than 1 found in Gamma_nj_k.


In [12]:
# Check 

beta = Gamma_nj.copy()  # beta is the variable name used in model simulation

# Check: Every value in beta is between 0 and 1
check_range = np.all((beta >= 0) & (beta <= 1))
if check_range:
    print("Every value in beta is between 0 and 1 ✅")
else:
    print("There are values in beta that are not between 0 and 1 ❌")
    print("These values are at positions:", np.where((beta < 0) | (beta > 1)))
    print("The values are:", beta[(beta < 0) | (beta > 1)])



Every value in beta is between 0 and 1 ✅


In [13]:
# Check 
# Check 1: Every value in gamma is between 0 and 1

gamma = Gamma_nj_k.reshape((N, J, J)) # gamma is the variable name used in model simulation:  $gamma(n, j, k)$ : country n, using sector j, producing sector k

invalid_values = (gamma < 0) | (gamma > 1)

if np.any(invalid_values):
    print("There are values in gamma that are not between 0 and 1 ❌")
    print("These values are at positions:", np.where(invalid_values))
    print("The values are:", gamma[invalid_values])
else:
    print("Every value in gamma is between 0 and 1 ✅")

# Check 2: sum(k) gamma[n,k,j] + beta[n,j] = 1
temp = np.sum(gamma, axis = 2) + beta
is_valid = np.allclose(temp, 1, atol = 1e-5)

if is_valid:
    print("Condition satisfied: sum(k) gamma[n, k, j] + beta[n, j] = 1 ✅")
else:
    print("Condition not satisfied: sum(k) gamma[n, k, j] + beta[n, j] ≠ 1 ❌")
    print("Positions where the condition fails:", np.where(~np.isclose(temp, 1, atol=1e-5)))
    print("Values that do not satisfy the condition:", temp[~np.isclose(temp, 1, atol=1e-5)])

Every value in gamma is between 0 and 1 ✅
Condition satisfied: sum(k) gamma[n, k, j] + beta[n, j] = 1 ✅


In [14]:
# Save to csv
gamma_njk_df = pd.DataFrame(Gamma_nj_k,
                            index=[f"{country}_{sector}"
                                for country in country_list
                                for sector  in sector_list],
                            columns=sector_list)

gamma_nj_df  = pd.DataFrame(Gamma_nj,
                            index=country_list,
                            columns=sector_list)

gamma_njk_df.to_csv(os.path.join(save_path, "Gamma_njk_2017.csv"))
gamma_nj_df.to_csv (os.path.join(save_path, "Gamma_nj_2017.csv"))

print("Gamma_njk_2017.csv and Gamma_nj_2017.csv saved successfully.")

Gamma_njk_2017.csv and Gamma_nj_2017.csv saved successfully.


## pif,pim,pit(N,N,J): Calibrate intermediate goods trade share $\pi_{ni}^{j,I}$, final goods trade share $\pi_{ni}^{j,F}$, and total trade share $\pi_{ni}^{j}$

pi(n,i,j) n is  importer, i is exporter, j is sector

In [15]:
# Sum all sectors within each country's usage; similar to fd_sum calculation
io_sum = io.reshape(N*J, N, J).sum(axis=2)

# check io_sum
has_neg_io  = np.any(io_sum < 0)
num_neg_io  = int((io_sum < 0).sum())
print(f"io_sum has negative? {has_neg_io}   # of negative: {num_neg_io}")

# check fd_sum
has_neg_fd  = np.any(fd_sum < 0)
num_neg_fd  = int((fd_sum < 0).sum())
print(f"fd_sum has negative? {has_neg_fd}   # of negative: {num_neg_fd}")

io_sum has negative? False   # of negative: 0
fd_sum has negative? False   # of negative: 0


In [16]:
# pi_intermediate 

B = io_sum.reshape(N, J, N).sum(axis=0)  #  (J, N) Sum over sectors ()

# Extend  B to (N*J, N) (you can view io_sum as N * (J*N) stacked vertically)
B_expanded = np.repeat(B[np.newaxis, :, :], N, axis=0).reshape(N*J, N)
pi_intermediate = io_sum / B_expanded


# check
below_zero = pi_intermediate< 0        
above_one  = pi_intermediate > 1       

num_below  = int(below_zero.sum())
num_above  = int(above_one.sum())

print(f" # of < 0 : {num_below}")
print(f" # of > 1 : {num_above}")

 # of < 0 : 0
 # of > 1 : 0


In [17]:
# pi_final
B_final = fd_sum.reshape(N, J, N).sum(axis=0)  # Shape (J, N)

B_final_expanded = np.repeat(B_final[np.newaxis, :, :], N, axis=0).reshape(N*J, N)
pi_final = fd_sum / B_final_expanded


# check
below_zero = pi_final < 0       
above_one  = pi_final  > 1       


num_below  = int(below_zero.sum())
num_above  = int(above_one.sum())


print(f" # of < 0 : {num_below}")
print(f" # of > 1 : {num_above}")

 # of < 0 : 0
 # of > 1 : 0


In [18]:
# pi_total
total_flow = io_sum  + fd_sum

B_total = total_flow.reshape(N, J, N).sum(axis=0)  


B_total_expanded = np.repeat(B_total[np.newaxis, :, :], N, axis=0).reshape(N*J, N)
pi_total = total_flow / B_total_expanded


# check
below_zero = pi_total < 0       
above_one  = pi_total  > 1        

num_below  = int(below_zero.sum())
num_above  = int(above_one.sum())


print(f" # of < 0 : {num_below}")
print(f" # of > 1 : {num_above}")


 # of < 0 : 0
 # of > 1 : 0


In [19]:
pim = pi_intermediate.reshape((N, N, J)) # pim is used in model simulation
pim = pim.swapaxes(0, 1)
pif = pi_final.reshape((N, N, J)) # pif is used in model simulation
pif = pif.swapaxes(0,1)
pit = pi_total.reshape((N, N, J)) # pit is used (may not used) in model simulation
pit = pit.swapaxes(0,1)


pi_sets = {
    "final": pif,
    "intermediate": pim,
    "all": pit
}

tol = 1e-5
for name, pi in pi_sets.items():
    temp = np.sum(pi, axis=1)
    is_valid = np.allclose(temp, 1, atol=tol)

    if is_valid:
        print(f"{name:>12} ✅ Consition satisfied: ∑₍i₎ π[{name}][n,i,j] = 1")
    else:
        bad = ~np.isclose(temp, 1, atol=tol)
        print(f"{name:>12} ❌ Consition not satisfied：there are {bad.sum()} positions not equal to 1")
        print("  Position (n,j)：", np.argwhere(bad))
        print("  Actual sum：", temp[bad])


       final ✅ Consition satisfied: ∑₍i₎ π[final][n,i,j] = 1
intermediate ✅ Consition satisfied: ∑₍i₎ π[intermediate][n,i,j] = 1
         all ✅ Consition satisfied: ∑₍i₎ π[all][n,i,j] = 1


In [20]:
# Save to csv
from Functions import  save_trade_share_long

# 1) Intermediate goods
save_trade_share_long(pi_intermediate,
                    country_list,
                    sector_list,
                    save_path,
                    "trade_share_inter_2017.csv")

# 2) Final goods
save_trade_share_long(pi_final,
                    country_list,
                    sector_list,
                    save_path,
                    "trade_share_final_2017.csv")

# 3) All goods
save_trade_share_long(pi_total,
                    country_list,
                    sector_list,
                    save_path,
                    "trade_share_total_2017.csv")


Saved trade-share file: /Users/lishuangcen/Dropbox/Tariff_Project/3_Result/parameters/trade_share_inter_2017.csv
Saved trade-share file: /Users/lishuangcen/Dropbox/Tariff_Project/3_Result/parameters/trade_share_final_2017.csv
Saved trade-share file: /Users/lishuangcen/Dropbox/Tariff_Project/3_Result/parameters/trade_share_total_2017.csv


,Importer,Exporter,Sector,Share
0,AUS,AUS,Agriculture,0.951798
1,AUS,AUS,Fishing,0.686534
2,AUS,AUS,Mining and Quarrying,0.925300
3,AUS,AUS,Food & Beverages,0.954165
4,AUS,AUS,Textiles and Wearing Apparel,0.912071
...,...,...,...,...
34220,ROW,ROW,Finacial Intermediation and Business Activities,0.788260
34221,ROW,ROW,Public Administration,0.992163
34222,ROW,ROW,"Education, Health and Other Services",0.327251
34223,ROW,ROW,Private Households,0.994939


## VA: shape (N, )

In [21]:
va_total.shape

va_matrix = va_total.reshape(N, J)   

VA = va_matrix.sum(axis=1)

df_va = pd.DataFrame({
    "Country": country_list,
    "ValueAdded": VA
})

df_va.to_csv("3_Result/parameters/VA.csv", index=False, encoding="utf-8")


# Check if every value in VA is greater than 0
check_positive = np.all(VA > 0)
if check_positive:
    print("Every country's value added is greater than 0 ✅")
else:
    print("There are values in VA that are less than or equal to 0 ❌")
    print("These values are at positions:", np.where(VA <= 0))
    print("The values are:", VA[VA <= 0])

Every country's value added is greater than 0 ✅


# Part II: Run this part after running "3_Estimate_TradeCost.ipynb"


## $\theta_n$

As that decided in "3_Estimate_TradeCost.ipynb"

In [22]:
# Trade elasticity: take from "Bolhuis, M. A., Chen, M. J., & Kett, B. R. (2023). Fragmentation in global trade: Accounting for commodities. International Monetary Fund."

# 1 Agriculture: 2.91
# 2 Fishing: 2.91                               
# 3 Mining and Quarrying: 3.41                               
# 4 Food & Beverages: 4.17                                     
# 5 Textiles and Wearing Apparel: 4.71
# 6 Wood and Paper: (8.8 + 8.21) /2 = 8.505                                                                    
# 7 Petroleum, Chemical and Non-Metallic Mineral Products:  (3.67 + 10.56 + 6.75 + 4.79)/4 = 6.4425
# 8 Metal Products: （7.39 + 4.22）/2 = 5.805
# 9 Electrical and Machinery:  (5.01 + 5.14 + 4.11) /3 = 4.753                                                      
# 10 Transport Equipment: (8.92 + 8.99)/2 = 8.955                               
# 11 Other Manufacturing: 4.06
# 12 Recycling: 4.06                
# ---------------------------------------------------------------------
# Services sectors: 8.35

theta = np.array([2.91, 2.91, 3.14, 4.17, 4.71, 8.505, 6.4425, 5.805, 4.753, 8.955, 4.06, 4.06,
                8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35])  



## $\tilde{\tau}$: shape $(N, N, J)$
The tariff rate vector, the $(n, i, j)$ denote the $(1+\tau)$ tariff rate of country $n$ on country $i$ for goods of sector $j$

In [23]:
# Import tariff data constructed in "3_Estimate_TradeCost.ipynb"
# Here I choose "tariff" column to apply to my model

tariff_all  = pd.read_csv(os.path.join(save_path, "All_Tariff_2017.csv"))
tariff_df  = tariff_all .iloc[:, 1:]
tariff_df = tariff_df[['Importer', 'Exporter', 'Sector', "tariff"]]

tariff_df['Exporter_Sector'] = tariff_df['Exporter'] + '_' + tariff_df['Sector']
tariff_df['Importer'] = pd.Categorical(
    tariff_df['Importer'],
    categories=tariff_df['Importer'].unique(),
    ordered=True
)
tariff_df['Exporter_Sector'] = pd.Categorical(
    tariff_df['Exporter_Sector'],
    categories=tariff_df['Exporter_Sector'].unique(),
    ordered=True
)

# pivot 并 reshape
tariff_matrix = tariff_df.pivot_table(
    index='Importer',
    columns='Exporter_Sector',
    values='tariff',
    aggfunc='first',
    observed=False
)
tariff_np   = tariff_matrix.to_numpy()
tariff_base = tariff_np.reshape((N, N, J))

# Construct tilde_tau
# tilde_tau[n,i,j]: country n's tariff on goods j from contry i (n:importer, i:exporte)
tilde_tau = tariff_base + 1


## Xf(N,J) and Xm(N,J): Calculate Expenditure for final goods and intermediate goods

This is not necessary for model simulation; just for check

In [24]:
# The observed data is basic price

# To obatin X in purchase price, we multiply by tau (1 + tariff)


Ym = io.reshape(N, J, N, J).sum(axis = 3) # (exporter, sector, importer)
# adjust by \tau
tau_esI = np.transpose(tilde_tau, (1, 2, 0))   

Xm_temp = Ym * tau_esI 
Xm  =  Xm_temp.sum(axis = 0 ).T   # (N,J)


Yf = fd_select.reshape(N,J,N,fd_n_2).sum(axis = 3) # (exporter, sector, importer)
Xf_temp = Yf  * tau_esI 
Xf  =  Xf_temp.sum(axis = 0 ).T # (N,J)

X = Xf + Xm  # (N,J)


# just for check
df = pd.DataFrame(X,
                index=country_list,
                columns=sector_list)
df.to_csv(os.path.join(wd, "check/Expenditure.csv"), index=True, encoding="utf-8")


In [25]:
# save to npz

np.savez('model_data_2017.npz',
        N = N,
        J = J,
        country_list = country_list, 
        sector_list = sector_list, 
        alpha = alpha, 
        beta = beta, 
        gamma = gamma, 
        theta = theta, 
        pif = pif, 
        pim = pim, 
        Xf = Xf,
        Xm = Xm,
        tilde_tau = tilde_tau,
        D = D, 
        VA = VA)



- 计算saving rate

- 全部改成tradable


- 检查Trade deficit， X的定义

